In [2]:
from __future__ import print_function, division

import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob

from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import time

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []
    i = 0
    for file in glob.glob("pokemon_data/*.mid"):
        midi = converter.parse(file)
        i = i + 1
        if i > 5: break
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 250

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        print('[train]: Starting...')
        time_start = time.time()
        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Insert saving; The generator image is saved every 500 steps
        save_interval = 500

        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)



            if (epoch + 1) % save_interval == 0:

                # Save the generator model in a periodic basis
                model_name = './pokemon_models/pokemon_' + str(epoch)
                self.generator.save(model_name + ".h5")

                # Generate a midi file in a periodic basis
                self.generate(notes, epoch)


            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [time: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss, time.time() - time_start))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes, epoch)
        self.plot_loss()


        time_end = time.time()
        print('[train]: Ended. Time elapsed: {}'.format(time_end - time_start))
        
    def generate(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        

        file_name = './pokemon_output/pokemon_' + str(epoch)
        create_midi(pred_notes, file_name)
        time_end = time.time()
        print('[generate midi]: Ended. Time elapsed: {}'.format(time_end - time_start))

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('./pokemon_output/GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=250)    
  gan.train(epochs=500, batch_size=32, sample_interval=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 250, 512)          1054720   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_8 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total para

65 [D loss: 0.369620, acc.: 81.25%] [G loss: 1.814921] [time: 20.663640]
66 [D loss: 0.316493, acc.: 93.75%] [G loss: 2.478543] [time: 20.883535]
67 [D loss: 0.293875, acc.: 89.06%] [G loss: 1.606921] [time: 21.106350]
68 [D loss: 0.275437, acc.: 89.06%] [G loss: 2.070954] [time: 21.327346]
69 [D loss: 0.305383, acc.: 90.62%] [G loss: 2.580058] [time: 21.549499]
70 [D loss: 0.168211, acc.: 90.62%] [G loss: 3.409539] [time: 21.771140]
71 [D loss: 0.151355, acc.: 93.75%] [G loss: 3.318336] [time: 21.992850]
72 [D loss: 0.434066, acc.: 81.25%] [G loss: 2.368668] [time: 22.216938]
73 [D loss: 0.282239, acc.: 92.19%] [G loss: 3.631080] [time: 22.440607]
74 [D loss: 0.211305, acc.: 92.19%] [G loss: 2.921126] [time: 22.665809]
75 [D loss: 0.361103, acc.: 85.94%] [G loss: 2.254417] [time: 22.885453]
76 [D loss: 0.745010, acc.: 67.19%] [G loss: 1.337765] [time: 23.104254]
77 [D loss: 0.307521, acc.: 89.06%] [G loss: 2.503303] [time: 23.322660]
78 [D loss: 0.393842, acc.: 85.94%] [G loss: 5.0088

177 [D loss: 0.522438, acc.: 73.44%] [G loss: 1.090229] [time: 45.327185]
178 [D loss: 0.535070, acc.: 71.88%] [G loss: 1.106449] [time: 45.546826]
179 [D loss: 0.553691, acc.: 70.31%] [G loss: 1.270765] [time: 45.767060]
180 [D loss: 0.525565, acc.: 76.56%] [G loss: 1.179613] [time: 45.988038]
181 [D loss: 0.601176, acc.: 65.62%] [G loss: 1.204076] [time: 46.206748]
182 [D loss: 0.577230, acc.: 67.19%] [G loss: 1.221340] [time: 46.429491]
183 [D loss: 0.455761, acc.: 78.12%] [G loss: 1.397746] [time: 46.649153]
184 [D loss: 0.550279, acc.: 75.00%] [G loss: 1.425408] [time: 46.868196]
185 [D loss: 0.695819, acc.: 60.94%] [G loss: 1.237570] [time: 47.090284]
186 [D loss: 0.653924, acc.: 64.06%] [G loss: 1.344671] [time: 47.311579]
187 [D loss: 0.619045, acc.: 71.88%] [G loss: 1.110773] [time: 47.531186]
188 [D loss: 0.578203, acc.: 70.31%] [G loss: 1.202684] [time: 47.752528]
189 [D loss: 0.648137, acc.: 67.19%] [G loss: 1.179920] [time: 47.973610]
190 [D loss: 0.686456, acc.: 62.50%] [

288 [D loss: 0.590490, acc.: 67.19%] [G loss: 1.325398] [time: 69.836192]
289 [D loss: 0.600388, acc.: 67.19%] [G loss: 1.416505] [time: 70.061983]
290 [D loss: 0.613318, acc.: 67.19%] [G loss: 1.348998] [time: 70.282127]
291 [D loss: 0.633641, acc.: 67.19%] [G loss: 0.869675] [time: 70.505130]
292 [D loss: 0.617417, acc.: 62.50%] [G loss: 0.803590] [time: 70.729576]
293 [D loss: 0.526967, acc.: 81.25%] [G loss: 0.981924] [time: 70.956621]
294 [D loss: 0.689710, acc.: 57.81%] [G loss: 1.184628] [time: 71.184073]
295 [D loss: 0.688711, acc.: 57.81%] [G loss: 1.100962] [time: 71.407768]
296 [D loss: 0.582364, acc.: 73.44%] [G loss: 1.212375] [time: 71.630146]
297 [D loss: 0.620190, acc.: 64.06%] [G loss: 1.216454] [time: 71.852887]
298 [D loss: 0.548070, acc.: 73.44%] [G loss: 1.313273] [time: 72.070159]
299 [D loss: 0.598910, acc.: 70.31%] [G loss: 1.477735] [time: 72.296732]
300 [D loss: 0.602167, acc.: 70.31%] [G loss: 1.183430] [time: 72.516881]
301 [D loss: 0.580516, acc.: 68.75%] [

399 [D loss: 0.695516, acc.: 51.56%] [G loss: 0.772787] [time: 94.404164]
400 [D loss: 0.711175, acc.: 53.12%] [G loss: 0.766322] [time: 94.629194]
401 [D loss: 0.697481, acc.: 57.81%] [G loss: 0.762312] [time: 94.847689]
402 [D loss: 0.705752, acc.: 45.31%] [G loss: 0.754389] [time: 95.070529]
403 [D loss: 0.710112, acc.: 42.19%] [G loss: 0.739235] [time: 95.293231]
404 [D loss: 0.703693, acc.: 51.56%] [G loss: 0.744321] [time: 95.515237]
405 [D loss: 0.688577, acc.: 56.25%] [G loss: 0.738477] [time: 95.738403]
406 [D loss: 0.682157, acc.: 60.94%] [G loss: 0.761345] [time: 95.959710]
407 [D loss: 0.706599, acc.: 39.06%] [G loss: 0.749514] [time: 96.185251]
408 [D loss: 0.719544, acc.: 43.75%] [G loss: 0.729430] [time: 96.409593]
409 [D loss: 0.700295, acc.: 46.88%] [G loss: 0.722947] [time: 96.626607]
410 [D loss: 0.695680, acc.: 57.81%] [G loss: 0.728070] [time: 96.849337]
411 [D loss: 0.703015, acc.: 51.56%] [G loss: 0.708546] [time: 97.071752]
412 [D loss: 0.686329, acc.: 57.81%] [

TypeError: generate() takes 2 positional arguments but 3 were given